In [1]:
from torch.utils.data import DataLoader
import transformers as T
import public_params as pp
import torch
import math
from model_cq import Model4QA
tokenizer = pp.tokenizer
from data_ft import data4finetuning
ft_dataset = data4finetuning()

d:\install\Anaconda\envs\py3713\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from scipy import integrate
import copy
import random
standard = 0.3
average = 0.9
normal = lambda x: (1/(standard*(2*math.pi)**0.5))*math.e**(-(x-average)**2/(2*standard**2))
def get_batch(sample):
    lengths = [len(each[0])+2 for each in sample]
    context = [each[0] for each in sample]
    question = [each[1] for each in sample]
    answer = [each[2] for each in sample]

    prob = [normal(i/(len(sample)+1)) for i in range(len(sample)+1)]
    s = sum(prob)
    prob = [each/s for each in prob]
    r = random.random()
    for i in range(len(prob)-1, -1, -1):
        psum = sum(prob[:i])
        if r >= psum:
            negative_num = i
            break
    
    sampled = random.sample(range(len(sample)), negative_num)
    shuffled = copy.copy(sampled)
    question = [question[i] if i not in sampled else question[shuffled[sampled.index(i)]] for i in range(len(question))]
    random.shuffle(shuffled)
    label = [1 for _ in range(len(sample))]
    for i in range(len(sampled)):
        if context[sampled[i]] != context[shuffled[i]]:
            label[sampled[i]] = 0
    
    cq = [[101]+c+[102]+q+[102] for c, q in zip(context, question)]
    maxlen = max([len(each) for each in cq])
    input_ids = torch.zeros((len(sample), maxlen), dtype=torch.int64)
    mask = torch.zeros((len(sample), maxlen), dtype=torch.int64)
    token_type_ids = torch.zeros((len(sample), maxlen), dtype=torch.int64)
    labels = torch.zeros((len(sample)), dtype=torch.int64)
    for i in range(len(sample)):
        labels[i] = label[i]
        for j in range(len(cq[i])):
            input_ids[i, j] = cq[i][j]
            mask[i, j] = 1
            if j < lengths[i]:
                token_type_ids[i, j] = 1
    return input_ids, mask, token_type_ids, labels
# x = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]
# x[:0]
# jf = 0
# for i in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]:
#     print(f'{i}\t{negative_rate(i)}')
#     jf += 0.1*negative_rate(i)
# print(jf)
# jf, err = integrate.quad(negative_rate, 0, 1)
# jf, err = integrate.quad(lambda x:negative_rate(x)/jf, 0, 1)
# jf

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
myModel = Model4QA()
# myModel = torch.nn.DataParallel(myModel, device_ids=[0])
myModel = myModel.to(device)

lr_max = 1e-5
lr_min = 1e-7

def cls_loss(scores, labels):
    # total_loss = 0
    # for i in range(scores.size(0)):
    #     total_loss += torch.nn.CrossEntropyLoss(scores[i], labels[i])
    total_loss = torch.nn.CrossEntropyLoss()(scores, labels)
    return total_loss

Some weights of the model checkpoint at hfl/chinese-pert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from tqdm import tqdm
print(' -- Start training -- ')
warmup = 1500
optim = torch.optim.Adam(myModel.parameters(), lr=1)
scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lambda cur_iter: ((cur_iter/warmup)*(lr_max-lr_min)+lr_min) if cur_iter <= warmup else \
                                (lr_max*((1-lr_min/lr_max)/2+((1-lr_min/lr_max)/2)*math.cos((cur_iter-warmup)/(4500-warmup)*math.pi))))
for epoch in range(5):
    
    dl = DataLoader(ft_dataset, batch_size=8, collate_fn=get_batch, shuffle=True)
    loop = tqdm(dl, leave=True)
    running_loss = 0.
    myModel.train()
    acc = 0.
    all = 0.
    for index, batch in enumerate(loop):
        input_ids, mask, token_type_ids, labels= batch
        input_ids, mask, token_type_ids, labels = input_ids.to(device), mask.to(device), token_type_ids.to(device), labels.to(device)
        optim.zero_grad()
        start, end, cls = myModel(input_ids, mask=mask, token_type_ids=token_type_ids)
        loss = cls_loss(cls, labels)
        loss.backward()
        optim.step()
        scheduler.step()
        running_loss += loss.item()

        am = torch.argmax(cls, dim=-1)
        for i in range(labels.size(0)):
            if am[i] == labels[i]:
                acc += 1
            all += 1
    
    torch.save(myModel, f'./saved_model/epoch{epoch}')
    print('\nepoch {}, Loss {:.4f}, acc {:.4f}'.format(
        epoch, running_loss / len(dl), acc / all))
        


 -- Start training -- 


100%|██████████| 6847/6847 [1:06:48<00:00,  1.71it/s]



epoch 0, Loss 0.6503, acc 0.6444


100%|██████████| 6847/6847 [55:43<00:00,  2.05it/s]  



epoch 1, Loss 0.6514, acc 0.6449


...


epoch 2, Loss 0.6482, acc 0.6499


100%|██████████| 6847/6847 [51:54<00:00,  2.20it/s]



epoch 3, Loss 0.6477, acc 0.6504


100%|██████████| 6847/6847 [51:54<00:00,  2.20it/s]



epoch 4, Loss 0.6539, acc 0.6398


In [5]:
random.sample(range(2), 5)

ValueError: Sample larger than population or is negative